## Car Damage Severity Classification

This notebook implements a deep learning pipeline based on the DINOv3 ConvNeXt-Small backbone to classify car damage severity into minor, moderate, and severe. The dataset is made by Prajwal Bhamere and comes from https://www.kaggle.com/datasets/prajwalbhamere/car-damage-severity-dataset/data.

### Dependencies

In [ ]:
import torch
from torch import nn
from torch.optim import AdamW
from torchvision import transforms
from torchvision.datasets import ImageFolder
from transformers import AutoImageProcessor, AutoModel
from torch.utils.data import DataLoader
from torch.nn import CrossEntropyLoss
from tqdm import tqdm

In [ ]:
pretrained_model_name = "facebook/dinov3-convnext-small-pretrain-lvd1689m"
processor = AutoImageProcessor.from_pretrained(pretrained_model_name)
base_model = AutoModel.from_pretrained(
    pretrained_model_name, 
    device_map="auto", 
)

## Wrapper Module
Let's make a wrapper module to wrap the original model so we can expose our own forward method to return just the logits

In [ ]:
class CarDamageClassifier(nn.Module):
    def __init__(self, backbone, feature_dim: int = 768, hidden_dim = 256, dropout = 0.3):
        """
        Creates a torch.nn.Module for our car damage classifier.
        
        Parameters:
            feature_dim: int = 768 - Dimension of the input feature (DINOv3-ConvNext is 768)
            hidden_dim: int = 256 - Hidden layer dimensions
            dropout: float = 0.3 - Dropout Rate
        """
        super().__init__()
        self.backbone = backbone
        self.head = nn.Sequential(
            nn.Linear(feature_dim, hidden_dim),
            nn.ReLU(inplace=True),
            nn.Dropout(dropout),
            nn.Linear(hidden_dim, 3),
        )

    def forward(self, images):
        features = self.backbone(images).pooler_output
        logits = self.head(features)
        return logits

In [ ]:
model = CarDamageClassifier(base_model, feature_dim=768)
for p in model.backbone.parameters():
    p.requires_grad = False

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

## Data Loading
Let's load our data in and apply some augmentations

### Augmentations

First, let's decide which augmentations we actually need.

For both training and validation, we'll resize to 256x256.

For training, we’ll choose a more aggressive approach.

* **Random Horizontal Flips**
* **Color Jitter** (brightness, contrast, saturation, hue) so it doesn’t overfit to a specific lighting.
* **Mild Gaussian blur or small rotations** to simulate motion blur / slight camera tilt.
* **Normalization using Imagenet mean and standard deviation**, matching what was used to pretrain the DINOv3 ConvNeXt backbone.

For validation, there are no augmentations.


In [ ]:
train_transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.ColorJitter(
        brightness=0.2,
        contrast=0.2,
        saturation=0.2,
        hue=0.05
    ),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225],
    ),
])

val_transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225],
    ),
])

### Data Loading
Now, we can make use of torch's `ImageFolder` to load our data in and automatically apply augmentations

In [ ]:
train_dataset = ImageFolder("../dataset/train", transform=train_transform)
val_dataset   = ImageFolder("../dataset/val",   transform=val_transform)

### Training Configuration


Now we can shift to training our model. We need to configure some of the backing training configurations, such as loss, learning rate, epochs, batch size, etc. 

Once we define our batch size, we can also create our dataloaders.

In [ ]:
epochs = 100
batch_size = 32
learning_rate = 1e-5
optimizer = AdamW(params=model.head.parameters(), lr=learning_rate, weight_decay=0.01)
loss = CrossEntropyLoss()

In [ ]:
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4)
val_loader   = DataLoader(val_dataset,   batch_size=batch_size, shuffle=False, num_workers=4)

Now we can train the model!

In [ ]:
for epoch in range(epochs):
    train_loss = 0.0
    correct = 0
    total = 0

    model.train()  # ensure training mode each epoch

    # ----- TRAINING -----
    for images, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs}"):
        images = images.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()

        # forward pass → logits
        logits = model(images)

        # compute loss
        computed_loss = loss(logits, labels)

        # backward + step
        computed_loss.backward()
        optimizer.step()

        # stats
        train_loss += computed_loss.item() * images.size(0)
        _, preds = torch.max(logits, dim=1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)

    avg_train_loss = train_loss / total
    train_acc = correct / total

    # ----- VALIDATION -----
    model.eval()  # Enter evaluation mode
    val_loss = 0.0
    val_correct = 0
    val_total = 0

    with torch.no_grad():
        for images, labels in tqdm(val_loader, desc=f"Validation Epoch {epoch+1}/{epochs}"):
            images = images.to(device)
            labels = labels.to(device)

            logits = model(images)
            computed_loss = loss(logits, labels)

            val_loss += computed_loss.item() * images.size(0)
            _, preds = torch.max(logits, dim=1)
            val_correct += (preds == labels).sum().item()
            val_total += labels.size(0)

    avg_val_loss = val_loss / val_total
    val_acc = val_correct / val_total

    print(
        f"Epoch [{epoch+1}/{epochs}] "
        f"Train Loss: {avg_train_loss:.4f} | Train Acc: {train_acc:.4f} "
        f"| Val Loss: {avg_val_loss:.4f} | Val Acc: {val_acc:.4f}"
    )